In [138]:
import nltk
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
russian_stopwords = stopwords.words("russian")
from sklearn.metrics.pairwise import cosine_similarity
# from pymystem3 import Mystem
# mystem = Mystem()
from string import punctuation
import gensim
from gensim.models import Word2Vec, word2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import corpora
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os
from gensim.test.utils import get_tmpfile
import numpy as np
import math
import scipy as sp
from scipy.sparse.linalg import svds
from gensim.models.word2vec import LineSentence
from gensim.test.utils import datapath
import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as utils
from torch.utils.data import Dataset, TensorDataset, DataLoader
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from nltk.stem.snowball import RussianStemmer, SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
%matplotlib inline

In [172]:
directory = "texts"

texts_all = {0: [], 1: [], 2: []}

vocab = set()
label_dict = {0: [1, 0, 0], 1: [0, 1, 0], 2: [0, 0, 1]}

all_data_X = []
all_data_Y = []

In [173]:
for root, dirs, files in os.walk(directory, topdown=False):
    for name in tqdm.tqdm_notebook(files):
        if name.endswith(".txt"):            
            file_object = open(os.path.join(root, name), encoding='utf-8')
            read_ = file_object.read()
            i = int(name[:4])
            year_cat = -1
            
            if i < 1917:
                year_cat = 0
            if i < 1990 and i >= 1917:
                year_cat = 1
            if i < 2020 and i >= 1990:
                year_cat = 2
           
            tokens = nltk.word_tokenize(read_.lower())
            tokens = [word for word in tokens if word.isalpha()]
            tokens = [token for token in tokens if token not in russian_stopwords and token != " " \
                          and token.strip() not in punctuation]
            vocab.update(tokens)
            new_file = []
            k = 1
            while (k*200 < len(tokens)):
                texts_all[year_cat].append(tokens[(k-1)*200:k*200])
                new_file.append(tokens[(k-1)*200:k*200])
                all_data_X.append(tokens[(k-1)*200:k*200])
                all_data_Y.append(label_dict[year_cat])
                k += 1

            fn = str(year_cat) + '.txt'    
            with open(os.path.join("train_test_", fn), 'a',  encoding='utf-8') as ff:
                for k in range(len(new_file)):
                    ff.write(" ".join(new_file[k]))
                    ff.write('\n')

The data in all three categories is almost of the same size, so the dataset is balanced.

In [174]:
print(len(texts_all[0]))
print(len(texts_all[1]))
print(len(texts_all[2]))

6597
6664
6289


In [192]:
sentences_0 = LineSentence('train_test_/0.txt')
sentences_1 = LineSentence('train_test_/1.txt')
sentences_2 = LineSentence('train_test_/2.txt')

In [193]:
model_0_300_10_5 = word2vec.Word2Vec(sentences_0, size=300, window=5, workers=4, min_count=10, iter=20, sg=1)
model_1_300_10_5 = word2vec.Word2Vec(sentences_1, size=300, window=5, workers=4, min_count=10, iter=20, sg=1)
model_2_300_10_5 = word2vec.Word2Vec(sentences_2, size=300, window=5, workers=4, min_count=10, iter=20, sg=1)

In [194]:
model_0_300_10_5.save("word2vec_0_300_10_5.model")
model_1_300_10_5.save("word2vec_1_300_10_5.model")
model_2_300_10_5.save("word2vec_2_300_10_5.model")

In [195]:
model_0_300_10_5.wv.most_similar("любить")

[('прощать', 0.388449102640152),
 ('жаловать', 0.3844766616821289),
 ('заблестели', 0.3702448010444641),
 ('ненавидеть', 0.3659781217575073),
 ('деток', 0.36531126499176025),
 ('уважать', 0.3592938780784607),
 ('раскаиваться', 0.34629762172698975),
 ('горды', 0.3427136242389679),
 ('целуй', 0.3401082158088684),
 ('перестанет', 0.3394628167152405)]

In [196]:
model_1_300_10_5.wv.most_similar("любить")

[('ебет', 0.47063326835632324),
 ('ненавидеть', 0.4672117233276367),
 ('жалеть', 0.4515785574913025),
 ('прощать', 0.4411221146583557),
 ('ничему', 0.42597222328186035),
 ('счастливыми', 0.42581260204315186),
 ('нищим', 0.42127496004104614),
 ('беречь', 0.4152146279811859),
 ('дурочка', 0.4144856631755829),
 ('глупее', 0.41256988048553467)]

In [197]:
model_2_300_10_5.wv.most_similar("любить")

[('смею', 0.4245392680168152),
 ('жалеть', 0.4179636836051941),
 ('желать', 0.41600197553634644),
 ('живется', 0.4051150679588318),
 ('скоты', 0.40088891983032227),
 ('упрямый', 0.39710813760757446),
 ('поверят', 0.39596107602119446),
 ('ощущать', 0.39469560980796814),
 ('пожертвовать', 0.39360636472702026),
 ('рабом', 0.3928634822368622)]